In [5]:
import sys
import os
# caution: path[0] is reserved for script path (or '' in REPL).
sys.path.insert(1, os.path.abspath('./../src'))


from cdflib.epochs_astropy import CDFAstropy as cdfepoch
import astropy.time
import data_loader
import datetime
import matplotlib.pyplot as plt
import numpy as np
import os_helper
import pandas as pd
import tqdm
import multiprocessing as mp

import IRBEM

import data_loader
import chorus_machine_learning_helper
import rbsp_chorus_tool

import importlib
importlib.reload(data_loader)
importlib.reload(rbsp_chorus_tool)
%matplotlib qt

In [41]:
#STAGE 0 DATA VERIFICATION FOR POES LSTAR CALCULATIONS

year = 1998
refs = np.load(fr"./../processed_data_chorus_neural_network/STAGE_0/MPE_DATA_PREPROCESSED_WITH_LSTAR/MPE_PREPROCESSED_DATA_T89_{year}_Test.npz", allow_pickle=True)

DATA = refs["DATA"].flatten()[0]
print(DATA)
SAT = DATA['n15']
print(type(SAT))

dt_for_all = np.array([datetime.datetime.fromtimestamp(t) for t in SAT["UNIX_TIME"]])[:408]
L = SAT["L"][:408]
valid_data = SAT["Lstar"] > 0

Lstar = SAT["Lstar"][valid_data]
dt_for_all = dt_for_all[valid_data]
L = L[valid_data]

plt.plot(dt_for_all, Lstar, label="L*", color = "red", marker="*")
plt.plot(dt_for_all, L, label = "IGRF Lm", color = "black", marker="*")
plt.ylabel("|L|")
plt.xlabel("Time")
plt.title("Some Orbits for NOAA 15 in 1998")
plt.legend()

plt.show()

{'n15': {'UNIX_TIME': array([8.99337638e+08, 8.99337654e+08, 8.99337670e+08, ...,
       9.15062372e+08, 9.15062388e+08, 9.15062404e+08]), 'BLC_Angle': array([60.0606569 , 60.22211631, 60.40531253, ..., 49.47817342,
       49.62623049, 49.7966486 ]), 'BLC_Flux': array([[1.37677305e+04, 9.73845659e+03, 6.84373393e+03, ...,
        2.24349889e+03, 1.50539311e+03, 9.91196794e+02],
       [1.54514233e+04, 1.02519394e+04, 6.79301765e+03, ...,
        1.95346492e+03, 1.28230903e+03, 8.38451713e+02],
       [1.30353254e+04, 8.68605184e+03, 5.78025291e+03, ...,
        1.68366880e+03, 1.10983673e+03, 7.28665827e+02],
       ...,
       [1.18595952e+00, 1.25027509e+00, 1.31661439e+00, ...,
        1.51755838e+00, 1.58062968e+00, 1.63832745e+00],
       [1.06218402e+00, 1.12195677e+00, 1.18419677e+00, ...,
        1.37558442e+00, 1.43558087e+00, 1.48988254e+00],
       [1.23398558e+00, 1.27713329e+00, 1.31910974e+00, ...,
        1.42517618e+00, 1.44842839e+00, 1.46231748e+00]]), 'MLT': array([1

In [ ]:
#Interface for stage 1, Designed to do a year at a time

year = 2019

In [ ]:
#Stage 1 RBSP Chorus Preprocessing, Obtains clean chorus amplitudes

start = datetime.datetime(year = year, month = 1, day = 1)
end = datetime.datetime(year = year + 1, month = 1, day = 1)

WNA_survey_a = data_loader.load_raw_data_from_config(id=["RBSP", "EMFISIS", "L4", "WNA_SURVEY"],
                                                     start=start,
                                                     end=end,
                                                     satellite="a")

WNA_survey_b = data_loader.load_raw_data_from_config(id=["RBSP", "EMFISIS", "L4", "WNA_SURVEY"],
                                                     start=start,
                                                     end=end,
                                                     satellite="b")

WFR_spectral_matrix_a = data_loader.load_raw_data_from_config(id=["RBSP", "EMFISIS", "L2", "WFR_SPECTRAL_MATRIX_DIAGONAL"],
                                                              start=start,
                                                              end=end,
                                                              satellite="a")

WFR_spectral_matrix_b = data_loader.load_raw_data_from_config(id=["RBSP", "EMFISIS", "L2", "WFR_SPECTRAL_MATRIX_DIAGONAL"],
                                                              start=start,
                                                              end=end,
                                                              satellite="b")

In [ ]:
mlt_A = WNA_survey_a["MLT"]
L_A = WNA_survey_a["L"]
epoch_A = WNA_survey_a["Epoch"]

time_A = astropy.time.Time(cdfepoch.to_datetime(epoch_A), format="datetime").utc

chorus_A = rbsp_chorus_tool.iterate_through_days_and_calculate_chorus_amplitudes(WNA_survey = WNA_survey_a,
                                                                                 WFR_spectral_matrix = WFR_spectral_matrix_a)


within_epoch_range_A = (start < time_A) & (time_A < end)
finite_chorus_A = np.isfinite(chorus_A)
#This line might not be necessary but we want to train on clean data, literally any np.nan will fuck it ALL up. Ill probably double check before training
all_positive_coordinates_A = (epoch_A > 0) & (mlt_A > 0) & (L_A > 0)

epoch_A = epoch_A[within_epoch_range_A & finite_chorus_A & all_positive_coordinates_A]
L_A = L_A[within_epoch_range_A & finite_chorus_A & all_positive_coordinates_A]
mlt_A = mlt_A[within_epoch_range_A & finite_chorus_A & all_positive_coordinates_A]
chorus_A = chorus_A[within_epoch_range_A & finite_chorus_A & all_positive_coordinates_A]


mlt_B = WNA_survey_b["MLT"]
L_B = WNA_survey_b["L"]
epoch_B = WNA_survey_b["Epoch"]
time_B = astropy.time.Time(cdfepoch.to_datetime(epoch_B), format="datetime").utc

chorus_B = rbsp_chorus_tool.iterate_through_days_and_calculate_chorus_amplitudes(WNA_survey = WNA_survey_b,
                                                                                 WFR_spectral_matrix = WFR_spectral_matrix_b)

within_epoch_range_B = (start < time_B) & (time_B < end)
finite_chorus_B = np.isfinite(chorus_B)
all_positive_coordinates_B = (epoch_B > 0) & (mlt_B > 0) & (L_B > 0)

epoch_B = epoch_B[within_epoch_range_B & finite_chorus_B & all_positive_coordinates_B]
L_B = L_B[within_epoch_range_B & finite_chorus_B & all_positive_coordinates_B]
mlt_B = mlt_B[within_epoch_range_B & finite_chorus_B & all_positive_coordinates_B]
chorus_B = chorus_B[within_epoch_range_B & finite_chorus_B & all_positive_coordinates_B]

print(epoch_A.shape)
print(chorus_A.shape)
print(L_A.shape)

print(epoch_B.shape)
print(chorus_B.shape)
print(L_B.shape)

In [ ]:
#Save the RBSP stage 1 data, might honestly only need one stage
np.savez(file = os.path.abspath(f"./../processed_data_chorus_neural_network/STAGE_1/RBSP_OBSERVED_CHORUS_{year}.npz"), 
         EPOCH_A = epoch_A, 
         MLT_A = mlt_A, 
         L_A = L_A, 
         CHORUS_A = chorus_A,
         EPOCH_B = epoch_B, 
         MLT_B = mlt_B, 
         L_B = L_B, 
         CHORUS_B = chorus_B)


In [ ]:
#Stage 2, clean then combine RBSP, OMNI, and POES Data and find conjunctions between RBSP and POES

MAX_L_DIFF = 0.10
MAX_MLT_DIFF = 1.5
MAX_T_DIFF_SEC = 60

L_SCALE = (1.0 / MAX_L_DIFF)**2
MLT_SCALE = (1.0 / MAX_MLT_DIFF)**2
TIME_SCALE = (1.0 / MAX_T_DIFF_SEC)**2

CONJUNCTIONS_TOTAL = []

for _year in range(2012, 2020, 1):
    
    print(f"Began processing year : {_year}")
    
    print(f"Began loading RBSP Data for year: {_year}")
    refs = np.load(f"./../processed_data_chorus_neural_network/STAGE_1/RBSP_OBSERVED_CHORUS_{_year}.npz")
    RBSP_A = {}
    RBSP_A["EPOCH"] = refs["EPOCH_A"]
    RBSP_A["MLT"] = refs["MLT_A"]
    RBSP_A["L"] = refs["L_A"]
    RBSP_A["CHORUS"] = refs["CHORUS_A"]
    
    RBSP_B = {}
    RBSP_B["EPOCH"] = refs["EPOCH_B"]
    RBSP_B["MLT"] = refs["MLT_B"]
    RBSP_B["L"] = refs["L_B"]
    RBSP_B["CHORUS"] = refs["CHORUS_B"]
    
    refs.close()
    
    RBSP_A["UNIX_TIME"] = cdfepoch.unixtime(RBSP_A["EPOCH"])
    RBSP_B["UNIX_TIME"] = cdfepoch.unixtime(RBSP_B["EPOCH"])
    
    order_A = np.argsort(RBSP_A["UNIX_TIME"])
    order_B = np.argsort(RBSP_B["UNIX_TIME"])
    
    RBSP_A["UNIX_TIME"] = RBSP_A["UNIX_TIME"][order_A]
    RBSP_A["EPOCH"] = RBSP_A["EPOCH"][order_A]
    RBSP_A["MLT"] = RBSP_A["MLT"][order_A]
    RBSP_A["L"] = RBSP_A["L"][order_A]
    RBSP_A["CHORUS"] = RBSP_A["CHORUS"][order_A]

    RBSP_B["UNIX_TIME"] = RBSP_B["UNIX_TIME"][order_B]
    RBSP_B["EPOCH"] = RBSP_B["EPOCH"][order_B]
    RBSP_B["MLT"] = RBSP_B["MLT"][order_B]
    RBSP_B["L"] = RBSP_B["L"][order_B]
    RBSP_B["CHORUS"] = RBSP_B["CHORUS"][order_B]
    
    RBSP = [RBSP_A, RBSP_B]
    print(f"RBSP Data loaded for year : {_year}")
    
    print(f"Began loading POES Data for year : {_year}")
    POES = []
    
    for SAT in ["m01", "m02", "m03", "n15", "n16", "n17", "n18", "n19"]:
        
        POES_sat_refs = data_loader.load_raw_data_from_config(id=["POES", "SEM", "MPE"],
                                                              satellite=SAT,
                                                              start=datetime.datetime(year=_year, month=1, day=1),
                                                              end=datetime.datetime(year=_year, month=12, day=31, hour=23, minute=59, second=59))
        
        if POES_sat_refs:
            
            
            #This was done cause I wanted to scale the MLT before cleaning but Im lazy
            if _year < 2014:
                POES_sat_refs["MLT"] = (POES_sat_refs["MLT"] / 360.0) * 24.0
            
            valid_times = np.isfinite(POES_sat_refs["time"]) & (0 < POES_sat_refs["time"])
            valid_BLC_Angle = np.isfinite(POES_sat_refs["BLC_Angle"]) & (0 < POES_sat_refs["BLC_Angle"])
            valid_BLC_Flux = np.all(np.isfinite(POES_sat_refs["BLC_Flux"][:, :8]), axis=1) & np.all((0 < POES_sat_refs["BLC_Flux"][:, :8]), axis=1)
            valid_MLT = np.isfinite(POES_sat_refs["MLT"]) & (0 < POES_sat_refs["MLT"]) & (POES_sat_refs["MLT"] < 24)
            valid_L = np.isfinite(POES_sat_refs["lValue"]) & (0 < POES_sat_refs["lValue"]) & (POES_sat_refs["lValue"] < 10)
            valid_points = valid_times & valid_BLC_Angle & valid_BLC_Flux & valid_MLT & valid_L
            
            if np.any(valid_points):
                
                POES_sat_refs["time"] = POES_sat_refs["time"][valid_points]
                POES_sat_refs["BLC_Angle"] = POES_sat_refs["BLC_Angle"][valid_points]
                POES_sat_refs["BLC_Flux"] = POES_sat_refs["BLC_Flux"][valid_points, :8]
                POES_sat_refs["MLT"] = POES_sat_refs["MLT"][valid_points]
                POES_sat_refs["lValue"] = POES_sat_refs["lValue"][valid_points]
            
                if _year < 2014:
                                    
                    POES_sat_refs["UNIX_TIME"] = cdfepoch.unixtime(POES_sat_refs["time"])
                else:
                    POES_sat_refs["UNIX_TIME"] = (POES_sat_refs["time"] / 1000)
                    
                #Sort them so assumptions for binary search are satisfied:
                order = np.argsort(POES_sat_refs["UNIX_TIME"])
                POES_sat_refs["time"] = POES_sat_refs["time"][order]
                POES_sat_refs["UNIX_TIME"] = POES_sat_refs["UNIX_TIME"][order]
                POES_sat_refs["BLC_Angle"] = POES_sat_refs["BLC_Angle"][order]
                POES_sat_refs["BLC_Flux"] = POES_sat_refs["BLC_Flux"][order, :]
                POES_sat_refs["MLT"] = POES_sat_refs["MLT"][order]
                POES_sat_refs["lValue"] = POES_sat_refs["lValue"][order]
                
                POES.append(POES_sat_refs)
    
    if not POES:
        print(f"No POES satellite coverage found for year : {_year}")
        print(f"SKIPPING YEAR : {_year}")
        continue
    
    print(f"Finished loading POES data for year : {_year}")
    
    print(f"Began loading SUPERMAG data for year : {_year}")
    SUPERMAG_df = pd.read_csv(f"./../processed_data_chorus_neural_network/SUPERMAG_SME/sme_{_year}.csv")
    SUPERMAG = {}
    
    valid_SME = np.isfinite(SUPERMAG_df["SME"]) & (0 < SUPERMAG_df["SME"])
    
    if not np.any(valid_SME):
        print(f"No valid SME for year : {_year}")
        print(f"SKIPPING YEAR : {_year}")
        continue
    
    SUPERMAG["SME"] = np.array(SUPERMAG_df["SME"][valid_SME])
    SUPERMAG["Date_UTC"] = np.array(SUPERMAG_df["Date_UTC"][valid_SME])
    SUPERMAG["UNIX_TIME"] = astropy.time.Time(SUPERMAG["Date_UTC"].astype(str), scale="utc", in_subfmt='date_hms').unix
    
    order = np.argsort(SUPERMAG["UNIX_TIME"])
    SUPERMAG["SME"] = SUPERMAG["SME"][order]
    SUPERMAG["Date_UTC"] = SUPERMAG["Date_UTC"][order]
    SUPERMAG["UNIX_TIME"] = SUPERMAG["UNIX_TIME"][order]
    
    print(f"Finished loading SUPERMAG data for year : {_year}")
    
    print(f"Began loading OMNI data for year : {_year}")
    OMNI_refs = data_loader.load_raw_data_from_config(id = ["OMNI", "ONE_MIN_RESOLUTION"], 
                                                 start = datetime.datetime(year=_year, month=1, day=1),
                                                 end = datetime.datetime(year=_year, month=12, day=31, hour=23, minute=59, second=59))
    OMNI = {}
    
    valid_times = np.isfinite(OMNI_refs["Epoch"]) & (0 < OMNI_refs["Epoch"])
    valid_AVG_B = np.isfinite(OMNI_refs["F"]) & (0 <= OMNI_refs["F"]) & (OMNI_refs["F"] < 9990)
    valid_FLOW_SPEED = np.isfinite(OMNI_refs["flow_speed"]) & (0 <= OMNI_refs["flow_speed"]) & (OMNI_refs["flow_speed"] < 99900)
    valid_PROTON_DENSITY = np.isfinite(OMNI_refs["proton_density"]) & (-900 <= OMNI_refs["proton_density"]) & (OMNI_refs["proton_density"] < 900)
    valid_SYM_H = np.isfinite(OMNI_refs["SYM_H"]) & (-99000 <= OMNI_refs["SYM_H"]) & (OMNI_refs["SYM_H"] < 99900)
    valid_points = valid_times & valid_AVG_B & valid_FLOW_SPEED & valid_PROTON_DENSITY & valid_SYM_H
    
    if(not np.any(valid_points)):
        print(f"No valid OMNI DATA for year : {_year}")
        print(f"SKIPPING YEAR : {_year}")
        continue
    
    OMNI["EPOCH"] = OMNI_refs["Epoch"][valid_points]
    OMNI["UNIX_TIME"] = cdfepoch.unixtime(OMNI_refs["Epoch"][valid_points])
    OMNI["AVG_B"] = OMNI_refs["F"][valid_points]
    OMNI["FLOW_SPEED"] = OMNI_refs["flow_speed"][valid_points]
    OMNI["PROTON_DENSITY"] = OMNI_refs["proton_density"][valid_points]
    OMNI["SYM_H"] = OMNI_refs["SYM_H"][valid_points]
    
    order = np.argsort(OMNI["UNIX_TIME"])
    OMNI["EPOCH"] = OMNI["EPOCH"][order]
    OMNI["UNIX_TIME"] = OMNI["UNIX_TIME"][order]
    OMNI["AVG_B"] = OMNI["AVG_B"][order]
    OMNI["FLOW_SPEED"] = OMNI["FLOW_SPEED"][order]
    OMNI["PROTON_DENSITY"] = OMNI["PROTON_DENSITY"][order]
    OMNI["SYM_H"] = OMNI["SYM_H"][order]
    
    print(f"Finished loading OMNI data for year : {_year}")
    
    print(f"Finding CONJUNCTIONS for year : {_year}")
    CONJUNCTIONS_YEAR = []
    for POES_SAT in POES:
        
        NUMBER_OF_RECORDS = len(POES_SAT["UNIX_TIME"])
        CONJUNCTIONS = []
        
        print(f"Number of records: {NUMBER_OF_RECORDS}")
                
        for T in tqdm.tqdm(range(NUMBER_OF_RECORDS)):
            
            UNIX_TIME = POES_SAT["UNIX_TIME"][T]
            L = POES_SAT["lValue"][T]
            MLT = POES_SAT["MLT"][T]
            FLUX_SPECTRUM = POES_SAT["BLC_Flux"][T, :]
                        
            for RBSP_PROBE in RBSP:
                
                TIME_RANGE = np.searchsorted(a = RBSP_PROBE["UNIX_TIME"], v = [(UNIX_TIME - MAX_T_DIFF_SEC), (UNIX_TIME + MAX_T_DIFF_SEC)])

                NUM_CANDIDATES = 0
                TOTAL_TIME = 0
                TOTAL_L = 0
                TOTAL_DEL_MLT = 0
                TOTAL_CHORUS = 0
                
                for POINT in range(TIME_RANGE[0], TIME_RANGE[1], 1):
                    
                    del_L = (L - RBSP_PROBE["L"][POINT])
                    del_MLT = np.min( [(max(MLT, RBSP_PROBE["MLT"][POINT]) -  min(MLT, RBSP_PROBE["MLT"][POINT])),
                                      ((24 - max(MLT, RBSP_PROBE["MLT"][POINT])) + (min(MLT, RBSP_PROBE["MLT"][POINT]) - 0))])
                    
                    if (del_L**2 < MAX_L_DIFF**2) and (del_MLT**2 < MAX_MLT_DIFF**2):
                                                
                        NUM_CANDIDATES += 1
                        TOTAL_TIME += RBSP_PROBE["UNIX_TIME"][POINT]
                        TOTAL_L += RBSP_PROBE["L"][POINT]
                        TOTAL_DEL_MLT += del_MLT
                        TOTAL_CHORUS += RBSP_PROBE["CHORUS"][POINT]
                        
                if NUM_CANDIDATES == 0:
                    continue
                
                TIME_RANGE = np.searchsorted(a = SUPERMAG["UNIX_TIME"], v = [(UNIX_TIME - MAX_T_DIFF_SEC), (UNIX_TIME + MAX_T_DIFF_SEC)])
                AVG_SME = np.nanmean(SUPERMAG["SME"][TIME_RANGE[0]:TIME_RANGE[1]])

                TIME_RANGE = np.searchsorted(a = OMNI["UNIX_TIME"], v = [(UNIX_TIME - MAX_T_DIFF_SEC), (UNIX_TIME + MAX_T_DIFF_SEC)])
                AVG_AVG_B = np.nanmean(OMNI["AVG_B"][TIME_RANGE[0]:TIME_RANGE[1]])
                AVG_FLOW_SPEED = np.nanmean(OMNI["FLOW_SPEED"][TIME_RANGE[0]:TIME_RANGE[1]])
                AVG_PROTON_DENSITY = np.nanmean(OMNI["PROTON_DENSITY"][TIME_RANGE[0]:TIME_RANGE[1]])
                AVG_SYM_H = np.nanmean(OMNI["SYM_H"][TIME_RANGE[0]:TIME_RANGE[1]])
                
                if np.isfinite(AVG_SME) & np.isfinite(AVG_AVG_B) & np.isfinite(AVG_FLOW_SPEED) & np.isfinite(AVG_PROTON_DENSITY) & np.isfinite(AVG_SYM_H):
                    
                    
                    CONJUNCTION =  [UNIX_TIME, 
                                    L, 
                                    MLT,
                                    *FLUX_SPECTRUM,
                                    TOTAL_TIME / NUM_CANDIDATES, #TIME
                                    TOTAL_L / NUM_CANDIDATES, #L
                                    0,
                                    TOTAL_DEL_MLT / NUM_CANDIDATES, #del_MLT
                                    TOTAL_CHORUS / NUM_CANDIDATES, #CHORUS
                                    AVG_SME, 
                                    AVG_AVG_B,
                                    AVG_FLOW_SPEED, 
                                    AVG_PROTON_DENSITY,
                                    AVG_SYM_H]
                                                
                    CONJUNCTIONS.append(CONJUNCTION)
        

        print(f"Number of conjunctions: {len(CONJUNCTIONS)}")
        
        CONJUNCTIONS_YEAR.extend(CONJUNCTIONS)
    
    
    CONJUNCTIONS_TOTAL.extend(CONJUNCTIONS_YEAR)
    
    print(f"Total number of conjunctions so far: {len(CONJUNCTIONS_TOTAL)}")

CONJUNCTIONS_TO_BE_SAVED = np.vstack(CONJUNCTIONS_TOTAL)

print(f"Conjunctions to be saved: {CONJUNCTIONS_TO_BE_SAVED.shape}")

np.savez(f"./../processed_data_chorus_neural_network/STAGE_2/v5a/CONJUNCTIONS_v5a.npz",
        CONJUNCTIONS=CONJUNCTIONS_TO_BE_SAVED)


In [ ]:
#Stage 3, Look at the data and make sure its good enough, then remove solar proton events
version = "v5a"

CONJUNCTIONS_REFS = np.load(f"./../processed_data_chorus_neural_network/STAGE_2/{version}/CONJUNCTIONS_{version}.npz")

CONJUNCTIONS_TESTING = CONJUNCTIONS_REFS["CONJUNCTIONS"]

CONJUNCTIONS_REFS.close()

In [ ]:

'''PERCENTAGES_COVERED = [25.7899, 
                       25.7872,
                       16.9213,
                       16.8872,
                       23.2865,
                       23.2174,
                       21.2469,
                       21.1556,
                       17.7844,
                       17.9721,
                       22.3004,
                       22.5147,
                       23.2432,
                       23.2491,
                       20.0671,
                       19.0985] these were for v2a'''

'''CONJUNCTION =  [UNIX_TIME, 
                    L, 
                    MLT,
                    *FLUX_SPECTRUM,
                    candidate[0], #TIME
                    candidate[1], #L
                    candidate[2], #MLT
                    candidate[3], #del_MLT
                    candidate[4], #CHORUS
                    AVG_SME, 
                    AVG_AVG_B,
                    AVG_FLOW_SPEED, 
                    AVG_PROTON_DENSITY,
                    AVG_SYM_H]'''

CONJUNCTIONS_POES_TIME = CONJUNCTIONS_TESTING[:, 0]
CONJUNCTIONS_POES_L = CONJUNCTIONS_TESTING[:, 1]
CONJUNCTIONS_POES_MLT = CONJUNCTIONS_TESTING[:, 2]
CONJUNCTIONS_POES_FLUX = CONJUNCTIONS_TESTING[:, 3:-10]
CONJUNCTIONS_RBSP_TIME = CONJUNCTIONS_TESTING[:, -10]
CONJUNCTIONS_RBSP_L = CONJUNCTIONS_TESTING[:, -9]
CONJUNCTIONS_RBSP_MLT = CONJUNCTIONS_TESTING[:, -8]
CONJUNCTIONS_RBSP_DEL_MLT = CONJUNCTIONS_TESTING[:, -7]
CONJUNCTIONS_RBSP_CHORUS = CONJUNCTIONS_TESTING[:, -6]
CONJUNCTIONS_AVG_SME = CONJUNCTIONS_TESTING[:, -5]
CONJUNCTIONS_AVG_AVG_B = CONJUNCTIONS_TESTING[:, -4]
CONJUNCTIONS_AVG_FLOW_SPEED = CONJUNCTIONS_TESTING[:, -3]
CONJUNCTIONS_AVG_PROTON_DENSITY = CONJUNCTIONS_TESTING[:, -2]
CONJUNCTIONS_AVG_SYM_H = CONJUNCTIONS_TESTING[:, -1]

with open(f"./../processed_data_chorus_neural_network/STAGE_2/{version}/CONJUNCTIONS_{version}.txt", "w") as f:
    f.write("\nConjunctions:\n")
    f.write(f"Number of conjunctions: {CONJUNCTIONS_TESTING.shape[0]} [#]\n")
    f.write(f"Minimum RBSP Time: {np.min(CONJUNCTIONS_RBSP_TIME)} [seconds since unix epoch]\n")
    f.write(f"Maximum RBSP Time: {np.max(CONJUNCTIONS_RBSP_TIME)} [seconds since unix epoch]\n")
    f.write(f"Minimum POES Time: {np.min(CONJUNCTIONS_POES_TIME)} [seconds since unix epoch]\n")
    f.write(f"Maximum POES Time: {np.max(CONJUNCTIONS_POES_TIME)} [seconds since unix epoch]\n")
    
    f.write(f"\nL:\n")
    f.write(f"Mean Difference: {np.mean(CONJUNCTIONS_POES_L - CONJUNCTIONS_RBSP_L)} [L]\n")
    f.write(f"Standard deviation of Difference {np.std(CONJUNCTIONS_POES_L - CONJUNCTIONS_RBSP_L)} [L]\n")
    f.write(f"Minimum Absolute Difference : {np.min(np.abs(CONJUNCTIONS_POES_L - CONJUNCTIONS_RBSP_L))} [L]\n")
    f.write(f"Maximum Absolute Difference : {np.max(np.abs(CONJUNCTIONS_POES_L - CONJUNCTIONS_RBSP_L))} [L]\n")

    f.write("\nMLT: \n")
    f.write(f"Mean Absolute Difference: {np.mean(CONJUNCTIONS_RBSP_DEL_MLT)} [MLT]\n")
    f.write(f"Standard deviation of Absolute Difference {np.std(CONJUNCTIONS_RBSP_DEL_MLT)} [MLT]\n")
    f.write(f"Minimum Absolute Difference : {np.min(np.abs(CONJUNCTIONS_RBSP_DEL_MLT))} [MLT]\n")
    f.write(f"Maximum Absolute Difference : {np.max(np.abs(CONJUNCTIONS_RBSP_DEL_MLT))} [MLT]\n")

    f.write(f"\nTime: \n")
    f.write(f"Mean Difference: {np.mean(CONJUNCTIONS_POES_TIME - CONJUNCTIONS_RBSP_TIME)} [s]\n")
    f.write(f"Standard deviation of Difference {np.std(CONJUNCTIONS_POES_TIME - CONJUNCTIONS_RBSP_TIME)} [s]\n")
    f.write(f"Minimum Absolute Difference : {np.min(np.abs(CONJUNCTIONS_POES_TIME - CONJUNCTIONS_RBSP_TIME))} [s]\n")
    f.write(f"Maximum Absolute Difference : {np.max(np.abs(CONJUNCTIONS_POES_TIME - CONJUNCTIONS_RBSP_TIME))} [s]\n")

    f.write(f"\nChorus: \n")
    f.write(f"Mean: {np.mean(CONJUNCTIONS_RBSP_CHORUS)} [pT]\n")
    f.write(f"Standard Deviation: {np.std(CONJUNCTIONS_RBSP_CHORUS)} [pT]\n")
    f.write(f"Minimum: {np.min(CONJUNCTIONS_RBSP_CHORUS)} [pT]\n")
    f.write(f"Maximum: {np.max(CONJUNCTIONS_RBSP_CHORUS)} [pT]\n")


    f.write(f"\nSME: \n")
    f.write(f"Mean: {np.mean(CONJUNCTIONS_AVG_SME)} [nT]\n")
    f.write(f"Standard Deviation: {np.std(CONJUNCTIONS_AVG_SME)} [nT]\n")
    f.write(f"Minimum: {np.min(CONJUNCTIONS_AVG_SME)} [nT]\n")
    f.write(f"Maximum: {np.max(CONJUNCTIONS_AVG_SME)} [nT]\n")

    f.write(f"\nAVG_B: \n")
    f.write(f"Mean: {np.mean(CONJUNCTIONS_AVG_AVG_B)} [nT]\n")
    f.write(f"Standard Deviation: {np.std(CONJUNCTIONS_AVG_AVG_B)} [nT]\n")
    f.write(f"Minimum: {np.min(CONJUNCTIONS_AVG_AVG_B)} [nT]\n")
    f.write(f"Maximum: {np.max(CONJUNCTIONS_AVG_AVG_B)} [nT]\n")

    f.write(f"\nFlow Speed: \n")
    f.write(f"Mean: {np.mean(CONJUNCTIONS_AVG_FLOW_SPEED)} [km/s]\n")
    f.write(f"Standard Deviation: {np.std(CONJUNCTIONS_AVG_FLOW_SPEED)} [km/s]\n")
    f.write(f"Minimum: {np.min(CONJUNCTIONS_AVG_FLOW_SPEED)} [km/s]\n")
    f.write(f"Maximum: {np.max(CONJUNCTIONS_AVG_FLOW_SPEED)} [km/s]\n")
    
    f.write(f"\nProton Density: \n")
    f.write(f"Mean: {np.mean(CONJUNCTIONS_AVG_PROTON_DENSITY)} [n/cc]\n")
    f.write(f"Standard Deviation: {np.std(CONJUNCTIONS_AVG_PROTON_DENSITY)} [n/cc]\n")
    f.write(f"Minimum: {np.min(CONJUNCTIONS_AVG_PROTON_DENSITY)} [n/cc]\n")
    f.write(f"Maximum: {np.max(CONJUNCTIONS_AVG_PROTON_DENSITY)} [n/cc]\n")
    
    f.write(f"\nSYM_H: \n")
    f.write(f"Mean: {np.mean(CONJUNCTIONS_AVG_SYM_H)} [nT]\n")
    f.write(f"Standard Deviation: {np.std(CONJUNCTIONS_AVG_SYM_H)} [nT]\n")
    f.write(f"Minimum: {np.min(CONJUNCTIONS_AVG_SYM_H)} [nT]\n")
    f.write(f"Maximum: {np.max(CONJUNCTIONS_AVG_SYM_H)} [nT]\n")


In [ ]:
plt.title("RBSP - Closest POES L Shell Comparison")
plt.xlabel("RBSP L-Shell")
plt.ylabel("Closest POES L-Shell")
plt.hlines(y = 4, xmin=1, xmax=7, color="black")
plt.vlines(x = 4, ymin=1, ymax=7, color="black")

plt.scatter(CONJUNCTIONS_RBSP_L, CONJUNCTIONS_POES_L)

print(f"Mean difference: {np.mean(CONJUNCTIONS_POES_L - CONJUNCTIONS_RBSP_L)}")
print(f"Standard deviation of difference {np.std(CONJUNCTIONS_POES_L - CONJUNCTIONS_RBSP_L)}")
print(f"Maximum difference : {np.max(CONJUNCTIONS_POES_L - CONJUNCTIONS_RBSP_L)}")


In [ ]:
plt.title("RBSP MLT - Closest POES MLT")
plt.xlabel("RBSP MLT")
plt.ylabel("Closest POES MLT")
plt.hlines(y = 12, xmin=0, xmax=25, color="black")
plt.vlines(x = 12, ymin=0, ymax=25, color="black")

plt.scatter(CONJUNCTIONS_RBSP_MLT, CONJUNCTIONS_POES_MLT)

print(f"Mean difference: {np.mean(CONJUNCTIONS_RBSP_DEL_MLT)}")
print(f"Standard deviation of difference {np.std(CONJUNCTIONS_RBSP_DEL_MLT)}")
print(f"Maximum difference : {np.max(CONJUNCTIONS_RBSP_DEL_MLT)}")

In [ ]:
#Stage 3 Continued, Removing solar proton events!

version = "v5a"

CONJUNCTIONS_REFS = np.load(f"./../processed_data_chorus_neural_network/STAGE_2/{version}/CONJUNCTIONS_{version}.npz")

CONJUNCTIONS = CONJUNCTIONS_REFS["CONJUNCTIONS"]

CONJUNCTIONS_REFS.close()

SOLAR_PROTON_EVENT_LIST = pd.read_csv(f"./../processed_data_chorus_neural_network/SOLAR_PROTON_EVENT_LIST_1976_2024.csv")


In [ ]:
'''CONJUNCTION =  [UNIX_TIME, 
                    L, 
                    MLT,
                    *FLUX_SPECTRUM,
                    candidate[0], #TIME
                    candidate[1], #L
                    candidate[2], #MLT
                    candidate[3], #del_MLT
                    candidate[4], #CHORUS
                    AVG_SME, 
                    AVG_AVG_B,
                    AVG_FLOW_SPEED, 
                    AVG_PROTON_DENSITY,
                    AVG_SYM_H]'''


order_to_sort_conjunctions = np.argsort(CONJUNCTIONS[:, 0]) #Sorted based on POES Conjunction time!
SORTED_CONJUNCTIONS = CONJUNCTIONS[order_to_sort_conjunctions, :]

print(f"Starting shape of conjunctions list: {SORTED_CONJUNCTIONS.shape}")

SORTED_POES_CONJUNCTION_TIMES = SORTED_CONJUNCTIONS[:, 0]

START_OF_SEP_EVENTS_UTC = SOLAR_PROTON_EVENT_LIST["START"]
END_OF_SEP_EVENTS_UTC = SOLAR_PROTON_EVENT_LIST["END"]
ZIPPED_EVENTS = list(zip(START_OF_SEP_EVENTS_UTC, END_OF_SEP_EVENTS_UTC))

print(f"Removing high energy solar proton events!")

for SEP_EVENT in tqdm.tqdm(range(len(ZIPPED_EVENTS))):
    
    START = ZIPPED_EVENTS[SEP_EVENT][0].strip()
    END = ZIPPED_EVENTS[SEP_EVENT][1].strip()
    
    START_YMDHMS = {'year': int(START[0:4]), 'month': int(START[5:7]), 'day': int(START[8:10]), 'hour': int(START[11:13]), 'minute': int(START[13:15]), 'second': 0}
    END_YMDHMS = {'year': int(END[0:4]), 'month': int(END[5:7]), 'day': int(END[8:10]), 'hour': int(END[11:13]), 'minute': int(END[13:15]), 'second': 0}
    
    START_UNIX = astropy.time.Time(START_YMDHMS, format="ymdhms", scale='utc').unix
    END_UNIX = astropy.time.Time(END_YMDHMS, format="ymdhms", scale='utc').unix

    RANGE_TO_REMOVE = np.searchsorted(a = SORTED_POES_CONJUNCTION_TIMES, v = [START_UNIX, END_UNIX])
    
    SORTED_CONJUNCTIONS = np.vstack((SORTED_CONJUNCTIONS[0:RANGE_TO_REMOVE[0], :], SORTED_CONJUNCTIONS[RANGE_TO_REMOVE[1]:, :]))

print(f"Finished removing high energy solar proton events!")

print(f"Saving!")

CLEANED_CONJUNCTIONS = SORTED_CONJUNCTIONS #Should be cleaned by now!

np.savez(f"./../processed_data_chorus_neural_network/STAGE_3/{version}/CLEANED_CONJUNCTIONS_{version}.npz",
        CONJUNCTIONS=CLEANED_CONJUNCTIONS)

CONJUNCTIONS_POES_TIME = CLEANED_CONJUNCTIONS[:, 0]
CONJUNCTIONS_POES_L = CLEANED_CONJUNCTIONS[:, 1]
CONJUNCTIONS_POES_MLT = CLEANED_CONJUNCTIONS[:, 2]
CONJUNCTIONS_POES_FLUX = CLEANED_CONJUNCTIONS[:, 3:-10]
CONJUNCTIONS_RBSP_TIME = CLEANED_CONJUNCTIONS[:, -10]
CONJUNCTIONS_RBSP_L = CLEANED_CONJUNCTIONS[:, -9]
CONJUNCTIONS_RBSP_MLT = CLEANED_CONJUNCTIONS[:, -8]
CONJUNCTIONS_RBSP_DEL_MLT = CLEANED_CONJUNCTIONS[:, -7]
CONJUNCTIONS_RBSP_CHORUS = CLEANED_CONJUNCTIONS[:, -6]
CONJUNCTIONS_AVG_SME = CLEANED_CONJUNCTIONS[:, -5]
CONJUNCTIONS_AVG_AVG_B = CLEANED_CONJUNCTIONS[:, -4]
CONJUNCTIONS_AVG_FLOW_SPEED = CLEANED_CONJUNCTIONS[:, -3]
CONJUNCTIONS_AVG_PROTON_DENSITY = CLEANED_CONJUNCTIONS[:, -2]
CONJUNCTIONS_AVG_SYM_H = CLEANED_CONJUNCTIONS[:, -1]

print(f"Creating documentation of dataset!")


with open(f"./../processed_data_chorus_neural_network/STAGE_3/{version}/CLEANED_CONJUNCTIONS_{version}.txt", "w") as f:
    f.write("\nConjunctions:\n")
    f.write(f"Number of conjunctions: {CLEANED_CONJUNCTIONS.shape[0]} [#]\n")
    f.write(f"Number lost from cleaning solar proton events: {CONJUNCTIONS.shape[0] - CLEANED_CONJUNCTIONS.shape[0]} [#]\n")
    f.write(f"Minimum RBSP Time: {np.min(CONJUNCTIONS_RBSP_TIME)} [seconds since unix epoch]\n")
    f.write(f"Maximum RBSP Time: {np.max(CONJUNCTIONS_RBSP_TIME)} [seconds since unix epoch]\n")
    f.write(f"Minimum POES Time: {np.min(CONJUNCTIONS_POES_TIME)} [seconds since unix epoch]\n")
    f.write(f"Maximum POES Time: {np.max(CONJUNCTIONS_POES_TIME)} [seconds since unix epoch]\n")
    
    f.write(f"\nL:\n")
    f.write(f"Mean Difference: {np.mean(CONJUNCTIONS_POES_L - CONJUNCTIONS_RBSP_L)} [L]\n")
    f.write(f"Standard deviation of Difference {np.std(CONJUNCTIONS_POES_L - CONJUNCTIONS_RBSP_L)} [L]\n")
    f.write(f"Minimum Absolute Difference : {np.min(np.abs(CONJUNCTIONS_POES_L - CONJUNCTIONS_RBSP_L))} [L]\n")
    f.write(f"Maximum Absolute Difference : {np.max(np.abs(CONJUNCTIONS_POES_L - CONJUNCTIONS_RBSP_L))} [L]\n")

    f.write("\nMLT: \n")
    f.write(f"Mean Absolute Difference: {np.mean(CONJUNCTIONS_RBSP_DEL_MLT)} [MLT]\n")
    f.write(f"Standard deviation of Absolute Difference {np.std(CONJUNCTIONS_RBSP_DEL_MLT)} [MLT]\n")
    f.write(f"Minimum Absolute Difference : {np.min(np.abs(CONJUNCTIONS_RBSP_DEL_MLT))} [MLT]\n")
    f.write(f"Maximum Absolute Difference : {np.max(np.abs(CONJUNCTIONS_RBSP_DEL_MLT))} [MLT]\n")

    f.write(f"\nTime: \n")
    f.write(f"Mean Difference: {np.mean(CONJUNCTIONS_POES_TIME - CONJUNCTIONS_RBSP_TIME)} [s]\n")
    f.write(f"Standard deviation of Difference {np.std(CONJUNCTIONS_POES_TIME - CONJUNCTIONS_RBSP_TIME)} [s]\n")
    f.write(f"Minimum Absolute Difference : {np.min(np.abs(CONJUNCTIONS_POES_TIME - CONJUNCTIONS_RBSP_TIME))} [s]\n")
    f.write(f"Maximum Absolute Difference : {np.max(np.abs(CONJUNCTIONS_POES_TIME - CONJUNCTIONS_RBSP_TIME))} [s]\n")

    f.write(f"\nChorus: \n")
    f.write(f"Mean: {np.mean(CONJUNCTIONS_RBSP_CHORUS)} [pT]\n")
    f.write(f"Standard Deviation: {np.std(CONJUNCTIONS_RBSP_CHORUS)} [pT]\n")
    f.write(f"Minimum: {np.min(CONJUNCTIONS_RBSP_CHORUS)} [pT]\n")
    f.write(f"Maximum: {np.max(CONJUNCTIONS_RBSP_CHORUS)} [pT]\n")


    f.write(f"\nSME: \n")
    f.write(f"Mean: {np.mean(CONJUNCTIONS_AVG_SME)} [nT]\n")
    f.write(f"Standard Deviation: {np.std(CONJUNCTIONS_AVG_SME)} [nT]\n")
    f.write(f"Minimum: {np.min(CONJUNCTIONS_AVG_SME)} [nT]\n")
    f.write(f"Maximum: {np.max(CONJUNCTIONS_AVG_SME)} [nT]\n")

    f.write(f"\nAVG_B: \n")
    f.write(f"Mean: {np.mean(CONJUNCTIONS_AVG_AVG_B)} [nT]\n")
    f.write(f"Standard Deviation: {np.std(CONJUNCTIONS_AVG_AVG_B)} [nT]\n")
    f.write(f"Minimum: {np.min(CONJUNCTIONS_AVG_AVG_B)} [nT]\n")
    f.write(f"Maximum: {np.max(CONJUNCTIONS_AVG_AVG_B)} [nT]\n")

    f.write(f"\nFlow Speed: \n")
    f.write(f"Mean: {np.mean(CONJUNCTIONS_AVG_FLOW_SPEED)} [km/s]\n")
    f.write(f"Standard Deviation: {np.std(CONJUNCTIONS_AVG_FLOW_SPEED)} [km/s]\n")
    f.write(f"Minimum: {np.min(CONJUNCTIONS_AVG_FLOW_SPEED)} [km/s]\n")
    f.write(f"Maximum: {np.max(CONJUNCTIONS_AVG_FLOW_SPEED)} [km/s]\n")
    
    f.write(f"\nProton Density: \n")
    f.write(f"Mean: {np.mean(CONJUNCTIONS_AVG_PROTON_DENSITY)} [n/cc]\n")
    f.write(f"Standard Deviation: {np.std(CONJUNCTIONS_AVG_PROTON_DENSITY)} [n/cc]\n")
    f.write(f"Minimum: {np.min(CONJUNCTIONS_AVG_PROTON_DENSITY)} [n/cc]\n")
    f.write(f"Maximum: {np.max(CONJUNCTIONS_AVG_PROTON_DENSITY)} [n/cc]\n")
    
    f.write(f"\nSYM_H: \n")
    f.write(f"Mean: {np.mean(CONJUNCTIONS_AVG_SYM_H)} [nT]\n")
    f.write(f"Standard Deviation: {np.std(CONJUNCTIONS_AVG_SYM_H)} [nT]\n")
    f.write(f"Minimum: {np.min(CONJUNCTIONS_AVG_SYM_H)} [nT]\n")
    f.write(f"Maximum: {np.max(CONJUNCTIONS_AVG_SYM_H)} [nT]\n")

print(f"Finished!")
print(f"Ending shape of conjunctions : {CLEANED_CONJUNCTIONS.shape}")

In [ ]:
#Stage 4, Create datasets used for training, testing, etc

version = "v5a"

CONJUNCTIONS_REFS = np.load(f"./../processed_data_chorus_neural_network/STAGE_3/{version}/CLEANED_CONJUNCTIONS_{version}.npz")

CONJUNCTIONS = CONJUNCTIONS_REFS["CONJUNCTIONS"]

CONJUNCTIONS_REFS.close()

In [ ]:
print(CONJUNCTIONS.shape)

CONJUNCTIONS_CHORUS = CONJUNCTIONS[:, -6]

CONJUNCTIONS_RBSP_TIME = CONJUNCTIONS[:, -10]

where_chorus_greater_zero = (0 < CONJUNCTIONS_CHORUS)

jan1_unix = astropy.time.Time({"year":2016, "month":1, "day":1, "hour":0, "minute":0, "second":0}, format="ymdhms", scale="utc").unix
apr1_unix = astropy.time.Time({"year":2016, "month":4, "day":1, "hour":0, "minute":0, "second":0}, format="ymdhms", scale="utc").unix

where_between_feb1_apr1_2013 = (jan1_unix < CONJUNCTIONS_RBSP_TIME) & (CONJUNCTIONS_RBSP_TIME < apr1_unix)

train_test_subset_selected = where_chorus_greater_zero & ~where_between_feb1_apr1_2013
validation_subset_selected = where_chorus_greater_zero & where_between_feb1_apr1_2013

print(f"Number of conjunctions between feb1 and apr1 2013: {np.count_nonzero(where_between_feb1_apr1_2013)}")
print(f"Number of conjunctions with non-zero chorus: {np.count_nonzero(where_chorus_greater_zero)}")

CONJUNCTIONS_POES_TIME = np.expand_dims(CONJUNCTIONS[train_test_subset_selected, 0], axis = 1)
CONJUNCTIONS_POES_L = np.expand_dims(CONJUNCTIONS[train_test_subset_selected, 1], axis = 1)
CONJUNCTIONS_POES_MLT = np.expand_dims(CONJUNCTIONS[train_test_subset_selected, 2], axis = 1)
CONJUNCTIONS_POES_FLUX = CONJUNCTIONS[train_test_subset_selected, 3:9]
CONJUNCTIONS_RBSP_TIME = np.expand_dims(CONJUNCTIONS[train_test_subset_selected, -10], axis = 1)
CONJUNCTIONS_RBSP_L = np.expand_dims(CONJUNCTIONS[train_test_subset_selected, -9], axis = 1)
CONJUNCTIONS_RBSP_MLT = np.expand_dims(CONJUNCTIONS[train_test_subset_selected, -8], axis = 1)
CONJUNCTIONS_RBSP_DEL_MLT = np.expand_dims(CONJUNCTIONS[train_test_subset_selected, -7], axis = 1)
CONJUNCTIONS_RBSP_CHORUS = np.expand_dims(CONJUNCTIONS[train_test_subset_selected, -6], axis = 1)
CONJUNCTIONS_AVG_SME = np.expand_dims(CONJUNCTIONS[train_test_subset_selected, -5], axis = 1)
CONJUNCTIONS_AVG_AVG_B = np.expand_dims(CONJUNCTIONS[train_test_subset_selected, -4], axis = 1)
CONJUNCTIONS_AVG_FLOW_SPEED = np.expand_dims(CONJUNCTIONS[train_test_subset_selected, -3], axis = 1)
CONJUNCTIONS_AVG_PROTON_DENSITY = np.expand_dims(CONJUNCTIONS[train_test_subset_selected, -2], axis = 1)
CONJUNCTIONS_AVG_SYM_H = np.expand_dims(CONJUNCTIONS[train_test_subset_selected, -1], axis = 1)

print(CONJUNCTIONS_RBSP_TIME.shape)
print(CONJUNCTIONS_RBSP_L.shape)
print(CONJUNCTIONS_RBSP_MLT.shape)
print(CONJUNCTIONS_RBSP_CHORUS.shape)
print(CONJUNCTIONS_POES_TIME.shape)
print(CONJUNCTIONS_POES_L.shape)
print(CONJUNCTIONS_POES_MLT.shape)
print(CONJUNCTIONS_RBSP_DEL_MLT.shape)
print(CONJUNCTIONS_POES_FLUX.shape)
print(CONJUNCTIONS_AVG_SME.shape)
print(CONJUNCTIONS_AVG_AVG_B.shape)
print(CONJUNCTIONS_AVG_FLOW_SPEED.shape)
print(CONJUNCTIONS_AVG_PROTON_DENSITY.shape)
print(CONJUNCTIONS_AVG_SYM_H.shape)

mean_L = np.nanmean(CONJUNCTIONS_POES_L)
std_L = np.std(CONJUNCTIONS_POES_L)

print(f"Mean L: {mean_L}, STD L: {std_L}")

mean_fluxes = np.expand_dims(np.nanmean(np.log(CONJUNCTIONS_POES_FLUX), axis = 0), axis=0)
std_fluxes = np.expand_dims(np.nanstd(np.log(CONJUNCTIONS_POES_FLUX), axis = 0), axis = 0)

print(f"Mean fluxes : {mean_fluxes.shape}")

mean_sme = np.nanmean(CONJUNCTIONS_AVG_SME)
std_sme = np.std(CONJUNCTIONS_AVG_SME)

mean_avg_b = np.nanmean(CONJUNCTIONS_AVG_AVG_B)
std_avg_b = np.std(CONJUNCTIONS_AVG_AVG_B)

mean_flow_speed = np.nanmean(CONJUNCTIONS_AVG_FLOW_SPEED)
std_flow_speed = np.std(CONJUNCTIONS_AVG_FLOW_SPEED)

mean_avg_proton_density = np.nanmean(CONJUNCTIONS_AVG_PROTON_DENSITY)
std_avg_proton_density = np.std(CONJUNCTIONS_AVG_PROTON_DENSITY)

mean_avg_sym_h = np.nanmean(CONJUNCTIONS_AVG_SYM_H)
std_avg_sym_h = np.std(CONJUNCTIONS_AVG_SYM_H)

FEATURES = np.hstack(((CONJUNCTIONS_POES_L - mean_L) / std_L, 
                      np.sin((CONJUNCTIONS_POES_MLT * 2 * np.pi) / 24.0), 
                      np.cos((CONJUNCTIONS_POES_MLT * 2 * np.pi) / 24.0),
                      ((np.log(CONJUNCTIONS_POES_FLUX) - mean_fluxes) / std_fluxes)[:, [0, -1]],
                      #(CONJUNCTIONS_AVG_SME - mean_sme)  / std_sme))
                      #(CONJUNCTIONS_AVG_AVG_B - mean_avg_b) / std_avg_b,
                      (CONJUNCTIONS_AVG_FLOW_SPEED - mean_flow_speed) / std_flow_speed))
                      #(CONJUNCTIONS_AVG_PROTON_DENSITY - mean_avg_proton_density) / std_avg_proton_density,
                      #(CONJUNCTIONS_AVG_SYM_H - mean_avg_sym_h) / std_avg_sym_h))
                      
#SMALL VALIDATION SET:
CONJUNCTIONS_POES_TIME_VALIDATION = np.expand_dims(CONJUNCTIONS[validation_subset_selected, 0], axis = 1)
CONJUNCTIONS_POES_L_VALIDATION = np.expand_dims(CONJUNCTIONS[validation_subset_selected, 1], axis = 1)
CONJUNCTIONS_POES_MLT_VALIDATION = np.expand_dims(CONJUNCTIONS[validation_subset_selected, 2], axis = 1)
CONJUNCTIONS_POES_FLUX_VALIDATION = CONJUNCTIONS[validation_subset_selected, 3:9]
CONJUNCTIONS_RBSP_TIME_VALIDATION = np.expand_dims(CONJUNCTIONS[validation_subset_selected, -10], axis = 1)
CONJUNCTIONS_RBSP_L_VALIDATION = np.expand_dims(CONJUNCTIONS[validation_subset_selected, -9], axis = 1)
CONJUNCTIONS_RBSP_MLT_VALIDATION = np.expand_dims(CONJUNCTIONS[validation_subset_selected, -8], axis = 1)
CONJUNCTIONS_RBSP_DEL_MLT_VALIDATION = np.expand_dims(CONJUNCTIONS[validation_subset_selected, -7], axis = 1)
CONJUNCTIONS_RBSP_CHORUS_VALIDATION = np.expand_dims(CONJUNCTIONS[validation_subset_selected, -6], axis = 1)
CONJUNCTIONS_AVG_SME_VALIDATION = np.expand_dims(CONJUNCTIONS[validation_subset_selected, -5], axis = 1)
CONJUNCTIONS_AVG_AVG_B_VALIDATION = np.expand_dims(CONJUNCTIONS[validation_subset_selected, -4], axis = 1)
CONJUNCTIONS_AVG_FLOW_SPEED_VALIDATION = np.expand_dims(CONJUNCTIONS[validation_subset_selected, -3], axis = 1)
CONJUNCTIONS_AVG_PROTON_DENSITY_VALIDATION = np.expand_dims(CONJUNCTIONS[validation_subset_selected, -2], axis = 1)
CONJUNCTIONS_AVG_SYM_H_VALIDATION = np.expand_dims(CONJUNCTIONS[validation_subset_selected, -1], axis = 1)

print(CONJUNCTIONS_RBSP_TIME_VALIDATION.shape)
print(CONJUNCTIONS_RBSP_L_VALIDATION.shape)
print(CONJUNCTIONS_RBSP_MLT_VALIDATION.shape)
print(CONJUNCTIONS_RBSP_CHORUS_VALIDATION.shape)
print(CONJUNCTIONS_POES_TIME_VALIDATION.shape)
print(CONJUNCTIONS_POES_L_VALIDATION.shape)
print(CONJUNCTIONS_POES_MLT_VALIDATION.shape)
print(CONJUNCTIONS_RBSP_DEL_MLT_VALIDATION.shape)
print(CONJUNCTIONS_POES_FLUX_VALIDATION.shape)
print(CONJUNCTIONS_AVG_SME_VALIDATION.shape)
print(CONJUNCTIONS_AVG_AVG_B_VALIDATION.shape)
print(CONJUNCTIONS_AVG_FLOW_SPEED_VALIDATION.shape)
print(CONJUNCTIONS_AVG_PROTON_DENSITY_VALIDATION.shape)
print(CONJUNCTIONS_AVG_SYM_H_VALIDATION.shape)


VALIDATION_FEATURES = np.hstack(((CONJUNCTIONS_POES_L_VALIDATION - mean_L) / std_L, 
                                np.sin((CONJUNCTIONS_POES_MLT_VALIDATION * 2 * np.pi) / 24.0), 
                                np.cos((CONJUNCTIONS_POES_MLT_VALIDATION * 2 * np.pi) / 24.0),
                                ((np.log(CONJUNCTIONS_POES_FLUX_VALIDATION) - mean_fluxes) / std_fluxes)[:, [0, -1]],
                                #(CONJUNCTIONS_AVG_SME_VALIDATION - mean_sme) / std_sme))
                                #(CONJUNCTIONS_AVG_AVG_B_VALIDATION - mean_avg_b) / std_avg_b,
                                (CONJUNCTIONS_AVG_FLOW_SPEED_VALIDATION - mean_flow_speed) / std_flow_speed))
                                #(CONJUNCTIONS_AVG_PROTON_DENSITY_VALIDATION - mean_avg_proton_density) / std_avg_proton_density,
                                #(CONJUNCTIONS_AVG_SYM_H_VALIDATION - mean_avg_sym_h) / std_avg_sym_h))
        
np.savez(f"./../processed_data_chorus_neural_network/STAGE_4/{version}/MODEL_READY_DATA_{version}.npz",
        FEATURES = FEATURES,
        LABELS = CONJUNCTIONS_RBSP_CHORUS,
        VALIDATION_FEATURES = VALIDATION_FEATURES,
        VALIDATION_LABELS = CONJUNCTIONS_RBSP_CHORUS_VALIDATION,
        TRAINING_MLT = CONJUNCTIONS_POES_MLT,
        MEAN_FLUXES = mean_fluxes,
        STD_FLUXES = std_fluxes,
        MEAN_SME = mean_sme,
        STD_SME = std_sme,
        MEAN_AVG_B = mean_avg_b,
        STD_AVG_B = std_avg_b,
        MEAN_FLOW_SPEED = mean_flow_speed,
        STD_FLOW_SPEED = std_flow_speed,
        MEAN_AVG_PROTON_DENSITY = mean_avg_proton_density,
        STD_AVG_PROTON_DENSITY = std_avg_proton_density,
        MEAN_AVG_SYM_H = mean_avg_sym_h,
        STD_AVG_SYM_H = std_avg_sym_h)